# Web Scraping for Indeed.com & Predicting Salaries

In this project, we will practice two major skills: collecting data by scraping a website and then building a binary classifier.

We are going to collect salary information on data science jobs in a variety of markets. Then using the location, title, and summary of the job we will attempt to predict the salary of the job. For job posting sites, this would be extraordinarily useful. While most listings DO NOT come with salary information (as you will see in this exercise), being to able extrapolate or predict the expected salaries from other listings can help guide negotiations.

Normally, we could use regression for this task; however, we will convert this problem into classification and use a random forest classifier, as well as another classifier of your choice; either logistic regression, SVM, or KNN. 

- **Question**: Why would we want this to be a classification problem?
- **Answer**: While more precision may be better, there is a fair amount of natural variance in job salaries - predicting a range be may be useful.

Therefore, the first part of the assignment will be focused on scraping Indeed.com. In the second, we'll focus on using listings with salary information to build a model and predict additional salaries.

### Scraping job listings from Indeed.com

We will be scraping job listings from Indeed.com using BeautifulSoup. Luckily, Indeed.com is a simple text page where we can easily find relevant entries.

First, look at the source of an Indeed.com page: (http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10")

Notice, each job listing is underneath a `div` tag with a class name of `result`. We can use BeautifulSoup to extract those. 

#### Set up a request (using requests) to the URL below. Use BeautifulSoup to parse the page and extract all results (HINT: Look for div tags with class name result)
The URL here has many query parameters
- q for the job search
- This is followed by "+20,000" to return results with salaries (or expected salaries >$20,000)
- l for a location
- start for what result number to start on

In [11]:
URL = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

In [5]:
import urllib
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import re

Let's look at one result more closely. A single result looks like
```JSON
<div class=" row result" data-jk="2480d203f7e97210" data-tn-component="organicJob" id="p_2480d203f7e97210" itemscope="" itemtype="http://schema.org/JobPosting">
<h2 class="jobtitle" id="jl_2480d203f7e97210">
<a class="turnstileLink" data-tn-element="jobTitle" onmousedown="return rclk(this,jobmap[0],1);" rel="nofollow" target="_blank" title="AVP/Quantitative Analyst">AVP/Quantitative Analyst</a>
</h2>
<span class="company" itemprop="hiringOrganization" itemtype="http://schema.org/Organization">
<span itemprop="name">
<a href="/cmp/Alliancebernstein?from=SERP&campaignid=serp-linkcompanyname&fromjk=2480d203f7e97210&jcid=b374f2a780e04789" target="_blank">
    AllianceBernstein</a></span>
</span>
<tr>
<td class="snip">
<nobr>$117,500 - $127,500 a year</nobr>
<div>
<span class="summary" itemprop="description">
C onduct quantitative and statistical research as well as portfolio management for various investment portfolios. Collaborate with Quantitative Analysts and</span>
</div>
</div>
</td>
</tr>
</table>
</div>
```

While this has some more verbose elements removed, we can see that there is some structure to the above:
- The salary is available in a nobr element inside of a td element with class='snip.
- The title of a job is in a link with class set to jobtitle and a data-tn-element="jobTitle.
- The location is set in a span with class='location'.
- The company is set in a span with class='company'.

## Write 4 functions to extract these items (one function for each): location, company, job title, and salary.¶
Example
```python
def extract_location_from_result(result):
    return result.find ...
```

##### - Make sure these functions are robust and can handle cases where the data/field may not be available.
>- Remember to check if a field is empty or None for attempting to call methods on it
>- Remember to use try/except if you anticipate errors.

- **Test** the functions on the results above and simple examples

In [13]:
### Rithika suggested I merge all of the find_alls into one large function, so the code below was created with her assistance. 

def parse(url):
    html = requests.get(url)
    soup = BeautifulSoup(html.content, 'html.parser', from_encoding="utf-8")
    df = pd.DataFrame(columns=["Title","Location","Company","Salary", "Synopsis"])
    for each in soup.find_all(class_= "result" ):
        try: 
            title = each.find(class_='jobtitle').text.replace('\n', '')
        except:
            title = 'None'
        try:
            location = each.find('span', {'class':"location" }).text.replace('\n', '')
        except:
            location = 'None'
        try: 
            company = each.find(class_='company').text.replace('\n', '')
        except:
            company = 'None'
        try:
            salary = each.find('span', {'class':'no-wrap'}).text
        except:
            salary = 'None'
        synopsis = each.find('span', {'class':'summary'}).text.replace('\n', '')
        df = df.append({'Title':title, 'Location':location, 'Company':company, 'Salary':salary, 'Synopsis':synopsis}, ignore_index=True)
    return df

In [14]:
parse(URL)

,Title,Location,Company,Salary,Synopsis
0,Data Scientist - Big Data & Analytics,"New York, NY 10154",KPMG,None,KPMG is currently seeking a Data Scientist - B...
1,Healthcare & Life Sciences - Data Scientist,"New York, NY 10154",KPMG,None,KPMG is currently seeking a Healthcare & Life ...
2,Entry Level – Research Analyst/Editor/Content ...,"New York, NY 10017 (Midtown area)","XG Consultants Group, Inc.",$15 an hour,Job Overview: XG Consultants Group is looking ...
3,Quantitative Analyst,"New York, NY",UBS,None,"Proficient using C++, VBA, R, SAS, SQL, Oracle..."
4,Data Science Intern - Fall 2017,"New York, NY 10016 (Gramercy area)",FactSet,None,"Scientist and engineer. Extract, transform, an..."
5,Data Science Intern - Fall 2017,"New York, NY 10016 (Gramercy area)",FactSet Research Systems,None,Just the right kind of work for a passionate d...
6,"Specialist, Advanced Insights - McKinsey Solut...","New York, NY 10022 (Midtown area)",McKinsey & Company,None,As one of the fastest-growing parts of our fir...
7,Machine Learning Engineer,"New York, NY",Indellient,None,"Big data and analytics, digital content delive..."
8,Machine Learning Engineer - KYC,"New York, NY",Bloomberg,None,Experience with Big data technologies such as ...
9,Quantitative Research Analyst,"New York, NY",Morningstar,None,Perform ad-hoc data cleaning and statistical a...


Now, to scale up our scraping, we need to accumulate more results. We can do this by examining the URL above.
- "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

There are two query parameters here we can alter to collect more results, the l=New+York and the start=10. The first controls the location of the results (so we can try a different city). The second controls where in the results to start and gives 10 results (thus, we can keep incrementing by 10 to go further in the list).
##### Complete the following code to collect results from multiple cities and starting points.
- Enter your city below to add it to the search
- Remember to convert your salary to U.S. Dollars to match the other cities if the currency is different

In [15]:
YOUR_CITY = 'Washington%2C+DC'

In [16]:
url_template = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l={}&start={}"
max_results_per_city = 2000 # Set this to a high-value (5000) to generate more results. 
# Crawling more results, will also take much longer. First test your code on a small number of results and then expand.
i = 0
results = []
df_more = pd.DataFrame(columns=["Title","Location","Company","Salary", "Synopsis"])
for city in set(['New+York', 'Chicago', 'San+Francisco', 'Austin', 'Seattle', 
    'Los+Angeles', 'Philadelphia', 'Atlanta', 'Dallas', 'Pittsburgh', 
    'Portland', 'Phoenix', 'Denver', 'Houston', 'Miami', YOUR_CITY, 
    'Charlottesville', 'Richmond', 'Baltimore', 'Harrisonburg', 'San+Antonio', 'San+Diego', 'San+Jose'
    'Austin', 'Jacksonville', 'Indianapolis', 'Columbus', 'Fort+Worth', 'Charlotte', 'Detroit', 'El+Paso', 
    'Memphis', 'Boston', 'Nashville', 'Louisville', 'Milwaukee', 'Las+Vegas', 'Albuquerque', 'Tucson', 
    'Fresno', 'Sacramento', 'Long+Beach', 'Mesa', 'Virginia+Beach', 'Norfolk', 'Atlanta', 'Colorado+Springs',
    'Raleigh', 'Omaha', 'Oakland', 'Tulsa', 'Minneapolis', 'Cleveland', 'Wichita', 'Arlington', 'New+Orleans', 
    'Bakersfield', 'Tampa', 'Honolulu', 'Anaheim', 'Aurora', 'Santa+Ana', 'Riverside', 'Corpus+Christi', 'Pittsburgh', 
    'Lexington', 'Anchorage', 'Cincinnati', 'Baton+Rouge', 'Chesapeake', 'Alexandria', 'Fairfax', 'Herndon',
    'Reston', 'Roanoke']):
    for start in range(0, max_results_per_city, 10):
        # Grab the results from the request (as above)
        url = url_template.format(city, start)
        # Append to the full set of results
        html = requests.get(url)
        soup = BeautifulSoup(html.content, 'html.parser', from_encoding="utf-8")
        for each in soup.find_all(class_= "result" ):
            try: 
                title = each.find(class_='jobtitle').text.replace('\n', '')
            except:
                title = None
            try:
                location = each.find('span', {'class':"location" }).text.replace('\n', '')
            except:
                location = None
            try: 
                company = each.find(class_='company').text.replace('\n', '')
            except:
                company = None
            try:
                salary = each.find('span', {'class':'no-wrap'}).text
            except:
                salary = None
            try:
                synopsis = each.find('span', {'class':'summary'}).text.replace('\n', '')
            except:
                synopsis = None
            df_more = df_more.append({'Title':title, 'Location':location, 'Company':company, 'Salary':salary, 'Synopsis':synopsis}, ignore_index=True)
            i += 1
            if i % 1000 == 0:  # Ram helped me build this counter to see how many. You can visibly see Ram's vernacular in the print statements.
                print('You have ' + str(i) + ' results. ' + str(df_more.dropna().drop_duplicates().shape[0]) + " of these aren't rubbish.")    

You have 1000 results. 0 of these aren't rubbish.
You have 2000 results. 0 of these aren't rubbish.
You have 3000 results. 2 of these aren't rubbish.
You have 4000 results. 2 of these aren't rubbish.
You have 5000 results. 2 of these aren't rubbish.
You have 6000 results. 2 of these aren't rubbish.
You have 7000 results. 2 of these aren't rubbish.
You have 8000 results. 4 of these aren't rubbish.
You have 9000 results. 41 of these aren't rubbish.
You have 10000 results. 41 of these aren't rubbish.
You have 11000 results. 45 of these aren't rubbish.
You have 12000 results. 52 of these aren't rubbish.
You have 13000 results. 58 of these aren't rubbish.
You have 14000 results. 61 of these aren't rubbish.
You have 15000 results. 69 of these aren't rubbish.
You have 16000 results. 69 of these aren't rubbish.
You have 17000 results. 73 of these aren't rubbish.
You have 18000 results. 94 of these aren't rubbish.
You have 19000 results. 94 of these aren't rubbish.
You have 20000 results. 94 of

You have 156000 results. 475 of these aren't rubbish.
You have 157000 results. 476 of these aren't rubbish.
You have 158000 results. 476 of these aren't rubbish.
You have 159000 results. 476 of these aren't rubbish.
You have 160000 results. 476 of these aren't rubbish.
You have 161000 results. 476 of these aren't rubbish.
You have 162000 results. 476 of these aren't rubbish.
You have 163000 results. 476 of these aren't rubbish.
You have 164000 results. 477 of these aren't rubbish.
You have 165000 results. 488 of these aren't rubbish.
You have 166000 results. 497 of these aren't rubbish.
You have 167000 results. 501 of these aren't rubbish.
You have 168000 results. 501 of these aren't rubbish.
You have 169000 results. 501 of these aren't rubbish.
You have 170000 results. 504 of these aren't rubbish.
You have 171000 results. 506 of these aren't rubbish.
You have 172000 results. 506 of these aren't rubbish.
You have 173000 results. 506 of these aren't rubbish.
You have 174000 results. 506

In [17]:
df_more.to_csv('Indeed_Project_3_df_more_long_not_cleaned.csv', encoding='utf-8')

In [261]:
df_more = pd.read_csv('/Users/aakashtandel/Desktop/Indeed_Project_3_df_more_long_not_cleaned.csv')

In [262]:
df_more.drop('Unnamed: 0', axis=1, inplace=True)

In [263]:
df_more.head()

,Title,Location,Company,Salary,Synopsis
0,Systems Administrator,"Forest, VA",Innerspec Technologies,NaN,Manage backup and restore services to ensure t...
1,Environmental Scientist,"Blacksburg, VA",CyberCoders,NaN,Environmental Scientist If you are an Environm...
2,Senior Manager,United States,Exponent,NaN,"Providing case management, data processing, an..."
3,Software Engineering Specialist,"Roanoke, VA 24019",General Electric,NaN,You will work with a group of energized and fo...
4,Sr Staff Software Engineer,"Roanoke, VA 24019",General Electric,NaN,"Architects, Data Scientists, Businesses & Prod..."


#### Use the functions you wrote above to parse out the 4 fields - location, title, company and salary. Create a dataframe from the results with those 4 columns.

Lastly, we need to clean up salary data. 

1. Only a small number of the scraped results have salary information - only these will be used for modeling.
1. Some of the salaries are not yearly but hourly or weekly, these will not be useful to us for now
1. Some of the entries may be duplicated
1. The salaries are given as text and usually with ranges.

#### Find the entries with annual salary entries, by filtering the entries without salaries or salaries that are not yearly (filter those that refer to hour or week). Also, remove duplicate entries

In [264]:
print (df_more.head())
print (df_more.shape)
print (df_more[df_more.Salary != 'None'].shape)
df_more = df_more[df_more.Salary != 'None'].drop_duplicates().dropna()
print (df_more.shape)

                             Title           Location                 Company  \
0            Systems Administrator         Forest, VA  Innerspec Technologies   
1          Environmental Scientist     Blacksburg, VA             CyberCoders   
2                   Senior Manager      United States                Exponent   
3  Software Engineering Specialist  Roanoke, VA 24019        General Electric   
4       Sr Staff Software Engineer  Roanoke, VA 24019        General Electric   

  Salary                                           Synopsis  
0    NaN  Manage backup and restore services to ensure t...  
1    NaN  Environmental Scientist If you are an Environm...  
2    NaN  Providing case management, data processing, an...  
3    NaN  You will work with a group of energized and fo...  
4    NaN  Architects, Data Scientists, Businesses & Prod...  
(199055, 5)
(199055, 5)
(581, 5)


In [265]:
df_more = df_more[df_more.Salary.str.contains("hour") == False]
df_more = df_more[df_more.Salary.str.contains("month") == False]
print (df_more.shape)
df_more.head()

(409, 5)


,Title,Location,Company,Salary,Synopsis
2724,Environmental Consultant,"Nashville, TN 37220",LP Environmental,"$50,000 - $100,000 a year",We are seeking a mid-level Environmental Scien...
8054,Data Scientist – Marketing Analytics,"Chicago, IL 60601 (Loop area)",Enterprise Select,"$125,000 a year",We are currently looking for an experienced Da...
8194,Research Analyst,"Chicago, IL",Illinois Criminal Justice Information Authority,"$55,000 - $60,000 a year",Produce analyses of crime trends and provide d...
8197,Research Analyst,"Chicago, IL 60606 (Loop area)",ICJIA,"$55,000 - $60,000 a year",(3) analyzing qualitative and quantitative dat...
8234,"Data Scientist (Python, R, AWS, Azure)","Chicago, IL",Workbridge Associates,"$85,000 - $125,000 a year",Our startup client working on risk and fraud d...


#### Write a function that takes a salary string and converts it to a number, averaging a salary range if necessary

In [266]:
def salary_stripper(dataframe, column):
    dataframe[str(column)] = dataframe[str(column)].replace({'\$':''}, regex = True)
    dataframe[str(column)].replace(regex=True,inplace=True,to_replace=r'\D',value=r' ')
    dataframe[str(column)] = dataframe[str(column)].str.replace(' ',',')
    dataframe = dataframe.join(dataframe[str(column)].str.split(',,,', 1, expand=True).rename(columns={0:'Low', 1:'High'}))
    dataframe['Low'] = dataframe['Low'].str.replace(',','')
    dataframe['Low'] = dataframe['Low'].astype('float64')
    dataframe.drop(str(column), axis=1, inplace=True)
    dataframe['High'] = dataframe['High'].str.replace(',','')
    dataframe['High'] = dataframe['High'].apply(pd.to_numeric)
    dataframe['Average'] = dataframe[['Low', 'High']].mean(axis=1)
    return dataframe

In [267]:
df_more = salary_stripper(df_more, 'Salary')

In [268]:
df_more[50:60]

,Title,Location,Company,Synopsis,Low,High,Average
17086,Senior Data Scientist Healthcare,"Philadelphia, PA",Harnham,Senior Data Scientist - Healthcare. Reporting ...,170000.0,NaN,170000.0
17252,Data Scientist,"Philadelphia, PA",Smith Hanley Associates,Data Scientist Qualifications:. Data Scientist...,120000.0,140000.0,130000.0
17287,Predictive Analytics (Machine Learning),"Wilmington, DE","Kennedy Unlimited Inc, Professional Staffing",Salary 130K to 140K We are assisting our Clien...,130000.0,140000.0,135000.0
17376,Data Scientist/Machine Learning Specialist,"Philadelphia, PA",Jobspring Partners,"A leading healthcare analytical team, one of t...",100000.0,160000.0,130000.0
17406,NMR Spectroscopist,"West Point, PA",EPM Scientific,Scientist | Structure Elucidation*. Excellent ...,115000.0,145000.0,130000.0
17421,Data Scientist/Optimization Engineer,"Philadelphia, PA",Jobspring Partners,In this role you will be extracting power plan...,100000.0,125000.0,112500.0
17451,"Research Scientist (Gene Therapy, Molecular Cl...","Philadelphia, PA",Apex Life Sciences,Analyze and interpret the results including st...,50000.0,NaN,50000.0
17459,Machine Learning Researcher,"Philadelphia, PA",Jobspring Partners,"Experience with Data Mining. This agency, spec...",100000.0,130000.0,115000.0
17467,Quant Research Analyst,"Philadelphia, PA",3coast,Experience in data and time series analysis st...,150000.0,250000.0,200000.0
17481,Quantitative Analyst,"Philadelphia, PA",Winston Fox,"In this role, you will examine global markets,...",100000.0,110000.0,105000.0


In [269]:
df_more = df_more.join(df_more['Location'].str.split(',', 1, expand=True).rename(columns={0:'City', 1:'State'}))

In [270]:
df_more.head()

,Title,Location,Company,Synopsis,Low,High,Average,City,State
2724,Environmental Consultant,"Nashville, TN 37220",LP Environmental,We are seeking a mid-level Environmental Scien...,50000.0,100000.0,75000.0,Nashville,TN 37220
8054,Data Scientist – Marketing Analytics,"Chicago, IL 60601 (Loop area)",Enterprise Select,We are currently looking for an experienced Da...,125000.0,NaN,125000.0,Chicago,IL 60601 (Loop area)
8194,Research Analyst,"Chicago, IL",Illinois Criminal Justice Information Authority,Produce analyses of crime trends and provide d...,55000.0,60000.0,57500.0,Chicago,IL
8197,Research Analyst,"Chicago, IL 60606 (Loop area)",ICJIA,(3) analyzing qualitative and quantitative dat...,55000.0,60000.0,57500.0,Chicago,IL 60606 (Loop area)
8234,"Data Scientist (Python, R, AWS, Azure)","Chicago, IL",Workbridge Associates,Our startup client working on risk and fraud d...,85000.0,125000.0,105000.0,Chicago,IL


In [271]:
def strip_state(x):
    if x != None:
        return x[0:3]
    else:
        None
df_more['State Initials'] = df_more['State'].apply(strip_state)
df_more.head()

,Title,Location,Company,Synopsis,Low,High,Average,City,State,State Initials
2724,Environmental Consultant,"Nashville, TN 37220",LP Environmental,We are seeking a mid-level Environmental Scien...,50000.0,100000.0,75000.0,Nashville,TN 37220,TN
8054,Data Scientist – Marketing Analytics,"Chicago, IL 60601 (Loop area)",Enterprise Select,We are currently looking for an experienced Da...,125000.0,NaN,125000.0,Chicago,IL 60601 (Loop area),IL
8194,Research Analyst,"Chicago, IL",Illinois Criminal Justice Information Authority,Produce analyses of crime trends and provide d...,55000.0,60000.0,57500.0,Chicago,IL,IL
8197,Research Analyst,"Chicago, IL 60606 (Loop area)",ICJIA,(3) analyzing qualitative and quantitative dat...,55000.0,60000.0,57500.0,Chicago,IL 60606 (Loop area),IL
8234,"Data Scientist (Python, R, AWS, Azure)","Chicago, IL",Workbridge Associates,Our startup client working on risk and fraud d...,85000.0,125000.0,105000.0,Chicago,IL,IL


### Save your results as a CSV

In [272]:
df_more.to_csv('Indeed_Project_3_df_cleaned.csv', encoding='utf-8')

## Predicting salaries using Random Forests + Another Classifier

#### Load in the the data of scraped salaries

In [404]:
df_more = pd.read_csv('/Users/aakashtandel/Desktop/Indeed_Project_3_df_cleaned.csv', index_col=0)
df_more = df_more.reset_index(drop=True)
df_more.head()

,Title,Location,Company,Synopsis,Low,High,Average,City,State,State Initials
0,Environmental Consultant,"Nashville, TN 37220",LP Environmental,We are seeking a mid-level Environmental Scien...,50000.0,100000.0,75000.0,Nashville,TN 37220,TN
1,Data Scientist – Marketing Analytics,"Chicago, IL 60601 (Loop area)",Enterprise Select,We are currently looking for an experienced Da...,125000.0,NaN,125000.0,Chicago,IL 60601 (Loop area),IL
2,Research Analyst,"Chicago, IL",Illinois Criminal Justice Information Authority,Produce analyses of crime trends and provide d...,55000.0,60000.0,57500.0,Chicago,IL,IL
3,Research Analyst,"Chicago, IL 60606 (Loop area)",ICJIA,(3) analyzing qualitative and quantitative dat...,55000.0,60000.0,57500.0,Chicago,IL 60606 (Loop area),IL
4,"Data Scientist (Python, R, AWS, Azure)","Chicago, IL",Workbridge Associates,Our startup client working on risk and fraud d...,85000.0,125000.0,105000.0,Chicago,IL,IL


#### We want to predict a binary variable - whether the salary was low or high. Compute the median salary and create a new binary variable that is true when the salary is high (above the median)

We could also perform Linear Regression (or any regression) to predict the salary value here. Instead, we are going to convert this into a _binary_ classification problem, by predicting two classes, HIGH vs LOW salary.

While performing regression may be better, performing classification may help remove some of the noise of the extreme salaries. We don't _have_ to choose the `median` as the splitting point - we could also split on the 75th percentile or any other reasonable breaking point.

In fact, the ideal scenario may be to predict many levels of salaries, 

In [405]:
median = df_more['Average'].median()
print ('The median salary for our data set is $' + str(median))

The median salary for our data set is $86711.0


In [406]:
def above_median(x):
    if x > median:
        return 1
    return 0

df_more['Above Median'] = df_more['Average'].apply(above_median)
df_more.head()

,Title,Location,Company,Synopsis,Low,High,Average,City,State,State Initials,Above Median
0,Environmental Consultant,"Nashville, TN 37220",LP Environmental,We are seeking a mid-level Environmental Scien...,50000.0,100000.0,75000.0,Nashville,TN 37220,TN,0
1,Data Scientist – Marketing Analytics,"Chicago, IL 60601 (Loop area)",Enterprise Select,We are currently looking for an experienced Da...,125000.0,NaN,125000.0,Chicago,IL 60601 (Loop area),IL,1
2,Research Analyst,"Chicago, IL",Illinois Criminal Justice Information Authority,Produce analyses of crime trends and provide d...,55000.0,60000.0,57500.0,Chicago,IL,IL,0
3,Research Analyst,"Chicago, IL 60606 (Loop area)",ICJIA,(3) analyzing qualitative and quantitative dat...,55000.0,60000.0,57500.0,Chicago,IL 60606 (Loop area),IL,0
4,"Data Scientist (Python, R, AWS, Azure)","Chicago, IL",Workbridge Associates,Our startup client working on risk and fraud d...,85000.0,125000.0,105000.0,Chicago,IL,IL,1


#### Thought experiment: What is the baseline accuracy for this model?

In [407]:
df_more['Above Median'].value_counts()

0    205
1    204
Name: Above Median, dtype: int64

Baseline accuracy for our model would be about 50/50 because if we flipped a coin to determine whether or not a value is above or below the median, we have a 50/50 chance. 

#### Create a Random Forest model to predict High/Low salary using Sklearn. Start by ONLY using the location as a feature. 

In [408]:
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
from ipywidgets import *
from IPython.display import display
from sklearn.cross_validation import cross_val_score, cross_val_predict
from sklearn import metrics
from sklearn.cross_validation import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier
from sklearn.cross_validation import train_test_split

In [409]:
df_city_dummy = pd.get_dummies(df_more['City'])
df_city_dummy.head()

,Albemarle County,Albuquerque,Anchorage,Arlington,Atlanta,Auburn,Aurora,Austin,Bakersfield,Baltimore,...,Tampa,Tampa Bay,Tempe,Tripler Army Medical Center,Tucson,Washington,West Point,Wichita,Wilmington,Woodlawn
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [410]:
df_state_dummy = pd.get_dummies(df_more['State Initials'])
df_state_dummy.head()

,AK,AZ,CA,CO,DC,DE,FL,GA,HI,IL,...,NV,NY,OH,OR,PA,TN,TX,VA,WA,WI
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [411]:
model = RandomForestClassifier(n_estimators=100, oob_score=True)
X = df_city_dummy
y = df_more['Above Median']
cv_model = cross_val_score(model, X, y, cv=6)
print 'Cross-validated scores:', cv_model
print 'Average score:', cv_model.mean()
print 'Standard deviation of score:', cv_model.std()
model.fit(X, y)
print (model.oob_score_)

Cross-validated scores: [ 0.50724638  0.38235294  0.33823529  0.55882353  0.58823529  0.51470588]
Average score: 0.481599886331
Standard deviation of score: 0.0908219073371
0.655256723716


It seems that the city locations are slightly better than our baseline accuracy of 50/50. 

#### Create a few new variables in your dataframe to represent interesting features of a job title.
- For example, create a feature that represents whether 'Senior' is in the title or whether 'Manager' is in the title. 
- Then build a new Random Forest with these features. Do they add any value?
- After creating these variables, use count-vectorizer to create features based on the words in the job titles.
- Build a new random forest model with location and these new features included.

In [412]:
def senior(x):
    if 'Senior' in x:
        return 1
    return 0

df_more['Senior'] = df_more['Title'].apply(senior)

In [413]:
df_more[df_more.Senior != 0]

,Title,Location,Company,Synopsis,Low,High,Average,City,State,State Initials,Above Median,Senior
19,Senior Data Scientist / Machine Learning Engineer,"Northbrook, IL",Request Technology,Seeking a Senior Data Scientist / Machine Lear...,130000.0,135000.0,132500.0,Northbrook,IL,IL,1,1
20,Senior Data Scientist,"Chicago, IL",Go2Group,Seeking a Data Scientist (Consultant to Archit...,130000.0,190000.0,160000.0,Chicago,IL,IL,1,1
24,Senior Data Scientist,"Chicago, IL",Workbridge Associates,You will be building out new predictive models...,130000.0,NaN,130000.0,Chicago,IL,IL,1,1
25,Senior Data Scientist - Modeling,"Chicago, IL",TS Kandilit,Sr Data Scientist – Modeling. Candidates with ...,100000.0,125000.0,112500.0,Chicago,IL,IL,1,1
29,Senior Data Engineer,"Chicago, IL",Jobspring Partners,Full stack data scientists looking to grow wit...,115000.0,130000.0,122500.0,Chicago,IL,IL,1,1
31,"Senior Data Engineer (AWS RedShift, MySQL, Pyt...","Chicago, IL",Workbridge Associates,Our client's Data Scientists and Engineers are...,80000.0,110000.0,95000.0,Chicago,IL,IL,1,1
38,Senior Data Scientist,"San Francisco, CA 94102 (Downtown area)",Aquila,Identify and own data latency and data quality...,100000.0,150000.0,125000.0,San Francisco,CA 94102 (Downtown area),CA,1,1
50,Senior Data Scientist Healthcare,"Philadelphia, PA",Harnham,Senior Data Scientist - Healthcare. Reporting ...,170000.0,NaN,170000.0,Philadelphia,PA,PA,1,1
65,Senior Statistician/Data Scientist-PHARMA,"Horsham, PA",Smith Hanley Associates,Statistician/Data Scientist Responsibilities. ...,120000.0,NaN,120000.0,Horsham,PA,PA,1,1
66,Senior Front-End Engineer,"Philadelphia, PA",Jobspring Partners,One will be a web-based application used by da...,135000.0,155000.0,145000.0,Philadelphia,PA,PA,1,1


In [414]:
def manager(x):
    if 'Manager' in x:
        return 1
    return 0

df_more['Manager'] = df_more['Title'].apply(manager)

In [415]:
df_more[df_more.Manager != 0]

,Title,Location,Company,Synopsis,Low,High,Average,City,State,State Initials,Above Median,Senior,Manager
43,District Sales Manager,"Dallas, TX",Anchor Point Technology Resources,"Ability to handle technical processes, data ga...",100000.0,NaN,100000.0,Dallas,TX,TX,1,0,1
79,"Laboratory Manager, IRL Blood Bank","Central, IN",Lighthouse Recruiting,Join the New Medical Laboratory Scientists Gro...,70000.0,87000.0,78500.0,Central,IN,IN,0,0,1
112,Computational Image Data Analysis Manager,"Bethesda, MD",Medical Science & Computing,Analysis of biopsy and resection samples from ...,100000.0,NaN,100000.0,Bethesda,MD,MD,1,0,1
125,Senior Product Manager,"Washington, DC",Catalist,Conversant understanding of relational data an...,70000.0,100000.0,85000.0,Washington,DC,DC,0,1,1
183,"2017-07-10 IT Program Manager Earning $$213,000","McLean, VA",EMF Industries,"Our talented group of Data Scientists, Develop...",213000.0,NaN,213000.0,McLean,VA,VA,1,0,1
199,Account Manager,"Charlotte, NC 28204 (Elizabeth area)",The Creative Group,You're fascinated by data analytics:. You may ...,50000.0,60000.0,55000.0,Charlotte,NC 28204 (Elizabeth area),NC,0,0,1
203,Analytics Manager (Machine Learning),"Charlotte, NC","Kennedy Unlimited Inc, Professional Staffing",8 Plus years of modeling experience in Big Dat...,120000.0,150000.0,135000.0,Charlotte,NC,NC,1,0,1
221,District Sales Manager,"Wichita, KS",Anchor Point Technology Resources,"Ability to handle technical processes, data ga...",100000.0,NaN,100000.0,Wichita,KS,KS,1,0,1
241,"Laboratory Manager, Pulmonary and Endothelial ...","Tucson, AZ",University of Arizona,"Carry out laboratory processing of samples, in...",60000.0,80000.0,70000.0,Tucson,AZ,AZ,0,0,1
254,"Accounting Manager (Phoenix, Arizona)","Tucson, AZ",University of Arizona,"Interprets various accounting data, analyzes r...",48645.0,74251.0,61448.0,Tucson,AZ,AZ,0,0,1


In [416]:
def engineer(x):
    if 'Engineer' in x:
        return 1
    return 0

df_more['Engineer'] = df_more['Title'].apply(engineer)

In [417]:
df_more[df_more.Engineer != 0]

,Title,Location,Company,Synopsis,Low,High,Average,City,State,State Initials,Above Median,Senior,Manager,Engineer
14,Data Scientist/Engineer,"Chicago, IL",Jobspring Partners,They would ideally have a STEM/Quantitative de...,70000.0,80000.0,75000.0,Chicago,IL,IL,0,0,0,1
19,Senior Data Scientist / Machine Learning Engineer,"Northbrook, IL",Request Technology,Seeking a Senior Data Scientist / Machine Lear...,130000.0,135000.0,132500.0,Northbrook,IL,IL,1,1,0,1
21,Data Engineer/Scientist,"Chicago, IL",Workbridge Associates,"Big Data experience with Hadoop, Spark, Pig. A...",90000.0,125000.0,107500.0,Chicago,IL,IL,1,0,0,1
22,Machine Learning Engineer (Ruby),"Chicago, IL",Jobspring Partners,Experience working with electronic medical rec...,110000.0,130000.0,120000.0,Chicago,IL,IL,1,0,0,1
26,Software Engineer (Python),"Chicago, IL",Jobspring Partners,Brand new product that is used globally by the...,75000.0,110000.0,92500.0,Chicago,IL,IL,1,0,0,1
29,Senior Data Engineer,"Chicago, IL",Jobspring Partners,Full stack data scientists looking to grow wit...,115000.0,130000.0,122500.0,Chicago,IL,IL,1,1,0,1
30,Civil/ Environmental Engineer,"Lombard, IL 60148","Andrews Engineering, Inc.",Analyzing Data or Information. Collaborate wit...,50000.0,65000.0,57500.0,Lombard,IL 60148,IL,0,0,0,1
31,"Senior Data Engineer (AWS RedShift, MySQL, Pyt...","Chicago, IL",Workbridge Associates,Our client's Data Scientists and Engineers are...,80000.0,110000.0,95000.0,Chicago,IL,IL,1,1,0,1
39,Sales Engineer,"San Francisco, CA",Pentaho a Hitachi Company,"ETL, data warehousing and data engineering. Co...",190000.0,200000.0,195000.0,San Francisco,CA,CA,1,0,0,1
49,Data Analytics Engineer,"Philadelphia, PA",CEI,Data Analytics Engineer*. An experienced data ...,85000.0,NaN,85000.0,Philadelphia,PA,PA,0,0,0,1


In [418]:
def data_scientist(x):
    if 'Data Scientist' in x:
        return 1
    return 0

df_more['Data Scientist'] = df_more['Title'].apply(data_scientist)

In [419]:
def analyst(x):
    if 'Analyst' in x:
        return 1
    return 0

df_more['Analyst'] = df_more['Title'].apply(analyst)

In [420]:
def assistant(x):
    if 'Assistant' in x:
        return 1
    return 0

df_more['Assistant'] = df_more['Title'].apply(assistant)

In [421]:
def internship(x):
    if 'Intern' in x:
        return 1
    return 0

df_more['Intern'] = df_more['Title'].apply(internship)

In [422]:
def developer(x):
    if 'Developer' in x:
        return 1
    return 0

df_more['Developer'] = df_more['Title'].apply(developer)

In [471]:
def machine_learning(x):
    if 'Machine Learning' in x:
        return 1
    return 0

df_more['Machine Learning'] = df_more['Title'].apply(machine_learning)

In [472]:
feature_matrix = df_more.copy(deep=True)
feature_matrix.drop(['Title', 'Location', 'Company', 'Synopsis', 'Low', 'High', 'Average', 'City', 'State', 'Above Median', 'State Initials'], axis=1, inplace=True)
print (feature_matrix.shape)
feature_matrix.head()

(409, 9)


,Senior,Manager,Engineer,Data Scientist,Analyst,Assistant,Intern,Developer,Machine Learning
0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0
3,0,0,0,0,1,0,0,0,0
4,0,0,0,1,0,0,0,0,0


In [473]:
model = RandomForestClassifier(n_estimators=100, oob_score=True)
X_features = pd.concat([df_city_dummy, feature_matrix, df_state_dummy], axis=1)
y = df_more['Above Median']
cv_model = cross_val_score(model, X_features, y, cv=6)
print 'Cross-validated scores:', cv_model
print 'Average score:', cv_model.mean()
print 'Standard deviation of score:', cv_model.std()
model.fit(X_features, y)
print (model.oob_score_)

Cross-validated scores: [ 0.71014493  0.61764706  0.52941176  0.76470588  0.61764706  0.61764706]
Average score: 0.642867291844
Standard deviation of score: 0.0754445265306
0.728606356968


Adding in the key words ('Senior', 'Manager', 'Data Scientist', 'Analyst', 'Assistant', 'Intern', and 'Developer) brought the accuracy up from  50% to around 63%. Let's attempt to increase this by using a CountVectorizer on the title column. 

In [474]:
from sklearn.feature_extraction.text import CountVectorizer
cvec = CountVectorizer(stop_words='english', max_features=30)
vectorizers = cvec.fit_transform(df_more['Title']).toarray()

df_vec  = pd.DataFrame(vectorizers, columns=cvec.get_feature_names())
print (df_vec.shape)
df_vec.head()

(409, 30)


,analyst,analytics,assistant,associate,bureau,clinical,data,director,engineer,environmental,...,python,research,science,scientist,senior,software,specialist,sr,statistical,statistician
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,...,1,0,0,1,0,0,0,0,0,0


In [475]:
X_cvec = pd.concat([df_vec, df_city_dummy, df_state_dummy], axis=1)
X_cvec.head()

,analyst,analytics,assistant,associate,bureau,clinical,data,director,engineer,environmental,...,NV,NY,OH,OR,PA,TN,TX,VA,WA,WI
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
1,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [484]:
model = RandomForestClassifier(n_estimators=100, oob_score=True)
y = df_more['Above Median']
cv_model = cross_val_score(model, X_cvec, y, cv=6)
print 'Cross-validated scores:', cv_model
print 'Average score:', cv_model.mean()
print 'Standard deviation of score:', cv_model.std()
model.fit(X_cvec, y)
print (model.oob_score_)
importance_dataframe = pd.DataFrame(model.feature_importances_, index = X_cvec.columns, columns=['importance']).sort_values('importance', ascending=False)
importance_dataframe.head(10)

Cross-validated scores: [ 0.73913043  0.67647059  0.64705882  0.77941176  0.58823529  0.57352941]
Average score: 0.667306052856
Standard deviation of score: 0.0744609682702
0.738386308068


,importance
data,0.081306
research,0.064954
scientist,0.048287
analyst,0.042719
engineer,0.020502
senior,0.018584
AZ,0.017347
CA,0.016853
sr,0.016421
learning,0.016151


Using the count vectorizer (and a max of 30 new word features) has increased our accuracy to around 67%. 

#### Use cross-validation in scikit-learn to evaluate the model above. 
- Evaluate the accuracy of the model, as well as any other metrics you feel are appropriate. 

The code above contains cross-validated accuracy scores for each model. The Random Forest Classifier with the description words Count Vectorized and the location dummies has an average accuracy score of 67%. 

#### Repeat the model-building process with a non-tree-based method.

In [428]:
from sklearn import linear_model
log_reg = linear_model.LogisticRegression()
scores_log = cross_val_score(log_reg, X_cvec, y, cv=6)
print 'Cross-validated scores:', scores_log
print 'Average score:', scores_log.mean()
print 'Standard deviation of score:', scores_log.std()
log_model = log_reg.fit(X_cvec, y)

Cross-validated scores: [ 0.71014493  0.64705882  0.52941176  0.79411765  0.61764706  0.63235294]
Average score: 0.655122193805
Standard deviation of score: 0.0817905778845


A logistic regression gets an average accuracy score of 65%, which isn't that much worse than our Random Forest Classifier. 

In [429]:
from sklearn.svm import SVC
model_svmrbf = SVC(kernel='rbf')
scores_svm = cross_val_score(model_svmrbf, X_cvec, y, cv=6)
print 'Cross-validated scores:', scores_svm
print 'Average score:', scores_svm.mean()
print 'Standard deviation of score:', scores_svm.std()
svm_model = model_svmrbf.fit(X_cvec, y)

Cross-validated scores: [ 0.76811594  0.64705882  0.55882353  0.77941176  0.73529412  0.64705882]
Average score: 0.689293833475
Standard deviation of score: 0.0785708648744


In [430]:
model_svmlm = SVC(kernel='linear')
scores_svmlm = cross_val_score(model_svmlm, X_cvec, y, cv=6)
print 'Cross-validated scores:', scores_svmlm
print 'Average score:', scores_svmlm.mean()
print 'Standard deviation of score:', scores_svmlm.std()
svm_model = model_svmlm.fit(X_cvec, y)

Cross-validated scores: [ 0.69565217  0.55882353  0.42647059  0.75        0.61764706  0.66176471]
Average score: 0.618393009378
Standard deviation of score: 0.104525736596


The radial basis kernel support vector machine performs better than the linear kernel support vector machine. The RBF SVM gets an accuracy score of 69%. This is higher than our Random Forest Classifier. 

In [478]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='sgd', alpha=1e-3, hidden_layer_sizes=(100,), random_state=1, activation='relu')
clf.fit(feature_matrix, y)
clf.score(feature_matrix, y)

0.68948655256723712

In [496]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0).fit(X_cvec, y)
cv_model = cross_val_score(clf, X_cvec, y, cv=6)
print 'Cross-validated scores:', cv_model
print 'Average score:', cv_model.mean()
print 'Standard deviation of score:', cv_model.std()               

Cross-validated scores: [ 0.73913043  0.58823529  0.47058824  0.73529412  0.67647059  0.64705882]
Average score: 0.642796248934
Standard deviation of score: 0.0927548377008


In [495]:
from xgboost import XGBClassifier
model = XGBClassifier(booster='gbtree')
cv_model = cross_val_score(model, X_cvec, y, cv=6)
print 'Cross-validated scores:', cv_model
print 'Average score:', cv_model.mean()
print 'Standard deviation of score:', cv_model.std()

Cross-validated scores: [ 0.76811594  0.75        0.63235294  0.77941176  0.61764706  0.63235294]
Average score: 0.696646774652
Standard deviation of score: 0.069896114897


### BONUS 

#### Bonus: Use Count Vectorizer from scikit-learn to create features from the job descriptions. 
- Examine using count or binary features in the model
- Re-evaluate your models using these. Does this improve the model performance? 
- What text features are the most valuable? 

In [431]:
cvec_synos = CountVectorizer(stop_words='english', max_features=50)
vectorizers_synos = cvec_synos.fit_transform(df_more['Synopsis']).toarray()
df_vec_synos  = pd.DataFrame(vectorizers_synos, columns=cvec_synos.get_feature_names())
print (df_vec_synos.shape)
df_vec_synos.head()

(409, 50)


,ability,analysis,analyst,analytics,analyze,analyzing,big,client,clinical,collection,...,senior,software,statistical,team,technical,use,using,work,working,years
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,1,1,0,0,0,0,...,0,1,1,0,0,0,2,0,0,0
4,0,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0


In [432]:
X_cvec_synos = pd.concat([df_vec, df_city_dummy, df_state_dummy, df_vec_synos], axis=1)
X_cvec_synos.head()

,analyst,analytics,assistant,associate,bureau,clinical,data,director,engineer,environmental,...,senior,software,statistical,team,technical,use,using,work,working,years
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
1,0,1,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,2,0,0,0
4,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0


In [433]:
X_cvec_synos.shape

(409, 236)

In [491]:
model = RandomForestClassifier(n_estimators=100, oob_score=True)
y = df_more['Above Median']
model.fit(X_cvec_synos, y)
print (model.oob_score_)
cv_model = cross_val_score(model, X_cvec_synos, y, cv=6)
print 'Cross-validated scores:', cv_model
print 'Average score:', cv_model.mean()
print 'Standard deviation of score:', cv_model.std()
importance_dataframe_big = pd.DataFrame(model.feature_importances_, index = X_cvec_synos.columns, columns=['importance']).sort_values('importance', ascending=False)
importance_dataframe_big.head(20)

0.753056234719
Cross-validated scores: [ 0.76811594  0.70588235  0.63235294  0.80882353  0.69117647  0.72058824]
Average score: 0.721156578573
Standard deviation of score: 0.0561385101897


,importance
research,0.049201
data,0.043424
data,0.035142
scientist,0.028058
analyst,0.024330
scientist,0.023333
scientists,0.020927
looking,0.018494
analysis,0.015077
DC,0.014487


After using the countvectorizer on the description, the accuracy of the model has increased from 70% to about 72%. That isn't a huge gain in accuracy and it is evident that the small sample size of 409 observations is hurting the model. 

In [402]:
from sklearn import linear_model
log_reg = linear_model.LogisticRegression()
log_model = log_reg.fit(X_cvec_synos, y)
scores_log = cross_val_score(log_reg, X_cvec_synos, y, cv=6)
print 'Cross-validated scores:', scores_log
print 'Average score:', scores_log.mean()
print 'Standard deviation of score:', scores_log.std()

Cross-validated scores: [ 0.75362319  0.67647059  0.54411765  0.75        0.66176471  0.64705882]
Average score: 0.672172492185
Standard deviation of score: 0.0705299844588


In [403]:
from sklearn.svm import SVC
model_svmrbf = SVC(kernel='rbf')
scores_svm = cross_val_score(model_svmrbf, X_cvec_synos, y, cv=6)
print 'Cross-validated scores:', scores_svm
print 'Average score:', scores_svm.mean()
print 'Standard deviation of score:', scores_svm.std()
svm_model = model_svmrbf.fit(X_cvec_synos, y)

Cross-validated scores: [ 0.82608696  0.66176471  0.63235294  0.76470588  0.66176471  0.75      ]
Average score: 0.716112531969
Standard deviation of score: 0.0689529908672


The RBF kernel SVM gets 72% accuracy. That is the highest we have seen so far. 